In [17]:
import pandas as pd

import plotly.express as px
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go

import nest_asyncio
from fastapi import FastAPI, Query
from fastapi.responses import HTMLResponse
import plotly.express as px
import uvicorn

# Appliquer nest_asyncio pour permettre l'exécution de uvicorn dans le notebook
nest_asyncio.apply()

In [10]:
df = pd.read_csv("Pokedex.csv")
df['IMC'] = df['Weight'] / (df['Height']) ** 2
df['Img'] = df['ID'].apply(lambda name: f'images/{name}.jpg' )
display(df.head())

,ID,Name,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Height,Weight,IMC,Img
0,1,Bulbizarre,Plante,Poison,318,45,49,49,65,65,45,1,0.7,6.9,14.081633,images/1.jpg
1,2,Herbizarre,Plante,Poison,405,60,62,63,80,80,60,1,1.0,13.0,13.000000,images/2.jpg
2,3,Florizarre,Plante,Poison,525,80,82,83,100,100,80,1,2.0,100.0,25.000000,images/3.jpg
3,4,Salamèche,Feu,NaN,309,39,52,43,60,50,65,1,0.6,8.5,23.611111,images/4.jpg
4,5,Reptincel,Feu,NaN,405,58,64,58,80,65,80,1,1.1,19.0,15.702479,images/5.jpg


Graphes

In [24]:
import nest_asyncio
from fastapi import FastAPI, Query
from fastapi.responses import HTMLResponse
import uvicorn
import pandas as pd
import plotly.express as px

nest_asyncio.apply()
app = FastAPI()

df = pd.read_csv("Pokedex.csv")
df['IMC'] = df['Weight'] / (df['Height']) ** 2
df['Img'] = df['ID'].apply(lambda name: f'images/{name}.jpg' )

@app.get("/", response_class=HTMLResponse)
async def index():
    # Générer la liste d'options pour le sélecteur multiple
    type_options = "".join(
        f'<option value="{t}">{t}</option>'
        for t in sorted(df["Type1"].unique())
    )
    
    html = f"""
    <html>
      <head>
        <title>Filtrer les Pokémon</title>
      </head>
      <body>
        <h1>Filtrer les Pokémon</h1>
        <!-- Conteneur flex pour tout aligner sur une même ligne -->
        <div style="display: flex; align-items: center; gap: 10px; flex-wrap: wrap;">
          <div>
            <label for="imc_min">IMC min :</label>
            <input type="number" id="imc_min" value="15" step="0.5" min="0" max="200">
          </div>
          <div>
            <label for="imc_max">IMC max :</label>
            <input type="number" id="imc_max" value="30" step="0.5" min="0" max="200">
          </div>
          <div>
            <label for="type_selector">Type1 :</label>
            <select id="type_selector" multiple size="5" style="width:120px;">
              {type_options}
            </select>
          </div>
          <button onclick="updatePlot()">Mettre à jour le graphique</button>
        </div>
        
        <!-- Zone où sera inséré le graphique -->
        <div id="plotDiv" style="margin-top:20px;"></div>
        
        <!-- Inclusion de Plotly.js -->
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <script>
          async function updatePlot() {{
            // Récupération des valeurs
            const imc_min = document.getElementById('imc_min').value;
            const imc_max = document.getElementById('imc_max').value;
            const typeSelect = document.getElementById('type_selector');
            let selected_types = [];
            for (let option of typeSelect.options) {{
              if(option.selected) {{
                selected_types.push(option.value);
              }}
            }}
            // Si aucun type n'est sélectionné, on prend tous les types
            if(selected_types.length === 0) {{
              for (let option of typeSelect.options) {{
                selected_types.push(option.value);
              }}
            }}

            // Construction de la requête
            const params = new URLSearchParams();
            params.append("imc_min_val", imc_min);
            params.append("imc_max_val", imc_max);
            for (const t of selected_types) {{
              params.append("selected_types", t);
            }}
            
            // Appel à l'endpoint qui renvoie le code HTML du graphique
            const response = await fetch("/update_plot?" + params.toString());
            const plotHTML = await response.text();
            document.getElementById("plotDiv").innerHTML = plotHTML;
          }}

          // Mettre à jour le graphique au chargement initial
          window.onload = updatePlot;
        </script>
      </body>
    </html>
    """
    return html

@app.get("/update_plot", response_class=HTMLResponse)
async def update_plot(
    selected_types: list[str] = Query(...),
    imc_min_val: float = 15,
    imc_max_val: float = 30
):
    # Filtrer selon les critères
    filtered_df = df[
        (df["Type1"].isin(selected_types)) &
        (df["IMC"] >= imc_min_val) &
        (df["IMC"] <= imc_max_val)
    ]
    # Créer le scatter plot
    if filtered_df.empty:
        fig = px.scatter(title="Aucun Pokémon ne correspond aux critères")
    else:
        fig = px.scatter(
            filtered_df,
            x="Weight",
            y="Height",
            hover_name="Name",
            hover_data={"IMC": True},
            color="Type1",
            trendline="lowess",
            title="Poids vs Taille (Filtré par Type1 et IMC)"
        )
    # Retourner le graphique au format HTML
    return fig.to_html(full_html=False, include_plotlyjs=False)

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")


Task exception was never retrieved
future: <Task finished name='Task-33' coro=<Server.serve() done, defined at c:\Users\thiev\anaconda3\Lib\site-packages\uvicorn\server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "c:\Users\thiev\anaconda3\Lib\site-packages\uvicorn\main.py", line 579, in run
    server.run()
  File "c:\Users\thiev\anaconda3\Lib\site-packages\uvicorn\server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\thiev\anaconda3\Lib\site-packages\nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\thiev\anaconda3\Lib\site-packages\nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "c:\Users\thiev\anaconda3\Lib\site-packages\nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "c:\Users\thiev\anaconda3\Lib\asyncio\events.py", line 88, 

INFO:     127.0.0.1:64324 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64324 - "GET /update_plot?imc_min_val=15&imc_max_val=30&selected_types=Acier&selected_types=Combat&selected_types=Dragon&selected_types=Eau&selected_types=Feu&selected_types=F%C3%A9e&selected_types=Glace&selected_types=Insecte&selected_types=Normal&selected_types=Plante&selected_types=Poison&selected_types=Psy&selected_types=Roche&selected_types=Sol&selected_types=Spectre&selected_types=T%C3%A9n%C3%A8bres&selected_types=Vol&selected_types=%C3%89lectrik HTTP/1.1" 200 OK
INFO:     127.0.0.1:64324 - "GET /update_plot?imc_min_val=15&imc_max_val=30&selected_types=Acier&selected_types=Combat&selected_types=Dragon&selected_types=Eau&selected_types=Feu&selected_types=F%C3%A9e&selected_types=Glace&selected_types=Insecte&selected_types=Normal&selected_types=Plante&selected_types=Poison&selected_types=Psy&selected_types=Roche&selected_types=Sol&selected_types=Spectre&selected_types=T%C3%A9n%C3%A8bres&selected_types=Vol&select

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [40368]


In [5]:
app = FastAPI()

@app.get("/", response_class=HTMLResponse)
async def afficher_graphiques():
    # Premier graphique : Scatter Plot avec le dataset Iris
    df1 = px.data.iris()
    fig1 = px.scatter(df1, x="sepal_width", y="sepal_length", color="species", title="Scatter Plot - Iris")
    
    # Deuxième graphique : Bar Chart avec le dataset Tips
    df2 = px.data.tips()
    fig2 = px.bar(df2, x="day", y="total_bill", color="sex", title="Bar Chart - Tips")
    
    # Convertir les figures en HTML
    # On inclut plotly.js une seule fois (avec include_plotlyjs='cdn')
    graph_html1 = fig1.to_html(include_plotlyjs='cdn', full_html=False)
    graph_html2 = fig2.to_html(full_html=False)  # pas besoin d'inclure plotly.js une deuxième fois

    html_content = f"""
    <html>
      <head>
        <title>Graphiques interactifs</title>
      </head>
      <body>
        <h1>Graphique 1 : Scatter Plot</h1>
        {graph_html1}
        <h1>Graphique 2 : Bar Chart</h1>
        {graph_html2}
      </body>
    </html>
    """
    return html_content

Task exception was never retrieved
future: <Task finished name='Task-10' coro=<Server.serve() done, defined at c:\Users\thiev\anaconda3\Lib\site-packages\uvicorn\server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "c:\Users\thiev\anaconda3\Lib\site-packages\uvicorn\main.py", line 579, in run
    server.run()
  File "c:\Users\thiev\anaconda3\Lib\site-packages\uvicorn\server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\thiev\anaconda3\Lib\site-packages\nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\thiev\anaconda3\Lib\site-packages\nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "c:\Users\thiev\anaconda3\Lib\site-packages\nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "c:\Users\thiev\anaconda3\Lib\asyncio\events.py", line 88, 

In [6]:
# Lancer le serveur FastAPI
uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

INFO:     Started server process [40368]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at c:\Users\thiev\anaconda3\Lib\site-packages\uvicorn\server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "c:\Users\thiev\anaconda3\Lib\site-packages\uvicorn\main.py", line 579, in run
    server.run()
  File "c:\Users\thiev\anaconda3\Lib\site-packages\uvicorn\server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\thiev\anaconda3\Lib\site-packages\nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\thiev\anaconda3\Lib\site-packages\nest_asyncio.py", line 92, in run_until_complete
  

INFO:     127.0.0.1:53904 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [40368]
